# PIPELINE MULTIPLE SIMULATION-K MEANS CLUSTERING

In [1]:
import json
import sys
import os
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
np.set_printoptions(precision=4)
sns.set()
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.cluster import KMeans
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

# Plotting functions

## Plot the positions of the finger

In [2]:
    def plot_pos_fingers(sides_pos,corners_pos,dim,pose,gap,trial):        
        SMALL_SIZE = 8
        MEDIUM_SIZE = 20
        BIGGER_SIZE = 30

        plt.rc('font', size=BIGGER_SIZE)          # controls default text sizes
        plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
        plt.rc('axes', labelsize=BIGGER_SIZE)    # fontsize of the x and y labels
        plt.rc('xtick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
        plt.rc('ytick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
        plt.rc('legend', fontsize=BIGGER_SIZE)    # legend fontsize
        plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

        fig, ax = plt.subplots()
        plt.style.use("ggplot")
        plt.rcParams["figure.figsize"] = (12,8)
        plt.scatter(sides_pos.T[0],sides_pos.T[2],color='b',marker='+',label='sides')
        plt.scatter(corners_pos.T[0],corners_pos.T[2],color='r',marker='+',label='corners')
        plt.ylabel(r'$\theta_R$')
        plt.xlabel(r'$\theta_L$')
        plt.legend(loc='lower right')
        plt.title("Raw data: dim="+str(dim)+'x'+str(dim)+ 'mm ; pose='+str(pose)+'mm ; gap='+str(gap)+'mm ; trial='+str(trial),fontsize=20)
        plt.savefig('raw_data_dim'+str(dim)+'x'+str(dim)+ '_pose_'+str(pose)+'_gap_'+str(gap)+'_'+str(trial)+'.png')

## Plot projection on feature vectors

In [3]:
def plot_proj_ld1(lda_X,y,dim,pose,gap,trial):
            SMALL_SIZE = 8
            MEDIUM_SIZE = 20
            BIGGER_SIZE = 30

            plt.rc('font', size=BIGGER_SIZE)          # controls default text sizes
            plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
            plt.rc('axes', labelsize=BIGGER_SIZE)    # fontsize of the x and y labels
            plt.rc('xtick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
            plt.rc('ytick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
            plt.rc('legend', fontsize=MEDIUM_SIZE)    # legend fontsize
            plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

            fig, ax = plt.subplots()
            plt.style.use("ggplot")
            plt.rcParams["figure.figsize"] = (12,8)
            ax.axes.yaxis.set_ticks([])
            plt.ylabel('LD1')
            plt.scatter(lda_X[y==0], np.full((len(lda_X[y==0]),),1.0), s=100, color='b', label='corner')
            plt.scatter(lda_X[y==1], np.full((len(lda_X[y==1]),),1.0), s=100, color='r', label='side')
            #plt.plot([-1.4,-1.4],[0.9,1.1],'--',color='g')
            plt.ylim(0.9,1.1)
            ax.legend()
            plt.title("1D LDA: dim="+str(dim)+'x'+str(dim)+ 'mm ; pose='+str(pose)+'mm ; gap='+str(gap)+'mm ; trial='+str(trial),fontsize=20)
            plt.savefig('1D_LDA_proj_dim'+str(dim)+'x'+str(dim)+ '_pose_'+str(pose)+'_gap_'+str(gap)+'_'+str(trial)+'.png')

## 2-means

In [4]:
def plot_ld1_2means(lda_X,y,y_k2means,dim,pose,gap,trial):
            SMALL_SIZE = 8
            MEDIUM_SIZE = 20
            BIGGER_SIZE = 30

            plt.rc('font', size=BIGGER_SIZE)          # controls default text sizes
            plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
            plt.rc('axes', labelsize=BIGGER_SIZE)    # fontsize of the x and y labels
            plt.rc('xtick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
            plt.rc('ytick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
            plt.rc('legend', fontsize=MEDIUM_SIZE)    # legend fontsize
            plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

            fig, ax = plt.subplots()
            plt.style.use("ggplot")
            plt.rcParams["figure.figsize"] = (12,8)
            ax.axes.yaxis.set_ticks([])
            plt.ylabel('LD1 & 2-means clustering')
            plt.scatter(lda_X[y==0], np.full((len(lda_X[y==0]),),2), color='orange', label='corner')
            plt.scatter(lda_X[y==1], np.full((len(lda_X[y==1]),),2), color='purple', label='side')
            plt.scatter(lda_X[y_k2means==0], np.full((len(lda_X[y_k2means==0]),),1.0), c='red', label ='Cluster 1')
            plt.scatter(lda_X[y_k2means==1], np.full((len(lda_X[y_k2means==1]),),1.0), c='blue', label ='Cluster 2')
            plt.scatter(k2means.cluster_centers_[:, 0], np.full((len(k2means.cluster_centers_[:, 0]),),1.0), 
                        s=100, c='yellow', marker='x',label = 'Centroids')
            #plt.plot([-1.4,-1.4],[0,4],'--',color='g')
            plt.ylim(0,4)
            ax.legend()
            plt.title("LD1 & 2-means: dim="+str(dim)+'x'+str(dim)+ 'mm ; pose='+str(pose)+'mm ; gap='+str(gap)+'mm ; trial='+str(trial),fontsize=20)
            plt.savefig('LD1_2_means_dim'+str(dim)+'x'+str(dim)+ '_pose_'+str(pose)+'_gap_'+str(gap)+'_'+str(trial)+'.png')
            #Plot the centroid. This time we're going to use the cluster centres  #attribute that returns here the coordinates of the centroid.
            
            ##########################################################################################

def plot_raw_2means(sides_pos,corners_pos,angle_array,half_window_size,y_k2means,dim,pose,gap,trial):
            SMALL_SIZE = 8
            MEDIUM_SIZE = 20
            BIGGER_SIZE = 30

            plt.rc('font', size=BIGGER_SIZE)          # controls default text sizes
            plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
            plt.rc('axes', labelsize=BIGGER_SIZE)    # fontsize of the x and y labels
            plt.rc('xtick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
            plt.rc('ytick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
            plt.rc('legend', fontsize=MEDIUM_SIZE)    # legend fontsize
            plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
            
            fig, ax = plt.subplots()
            plt.style.use("ggplot")
            plt.rcParams["figure.figsize"] = (12,8)
            plt.scatter(sides_pos.T[0],sides_pos.T[2]+10,
                        color='purple',marker='+',label='sides')
            plt.scatter(corners_pos.T[0],corners_pos.T[2]+10,
                        color='orange',marker='+',label='corners')
            plt.scatter(angle_array[half_window_size:-half_window_size].T[0][y_k2means == 0], 
                        angle_array[half_window_size:-half_window_size].T[2][y_k2means == 0]-10, 
                        marker='+', c='red', label ='Cluster 1')
            plt.scatter(angle_array[half_window_size:-half_window_size].T[0][y_k2means == 1], 
                        angle_array[half_window_size:-half_window_size].T[2][y_k2means == 1]-10, 
                        marker='+', c='blue', label ='Cluster 2')
            plt.plot(angle_array[half_window_size:-half_window_size].T[0],
                    angle_array[half_window_size:-half_window_size].T[2],
                    color='yellow', label='raw data')
            plt.ylabel(r'$\theta_R$')
            plt.xlabel(r'$\theta_L$')
            plt.legend(loc='lower right')
            plt.title("raw data processed (2-means): dim="+str(dim)+'x'+str(dim)+ 'mm ; pose='+str(pose)+'mm ; gap='+str(gap)+'mm ; trial='+str(trial),fontsize=20)
            plt.savefig('clusters_class_raw_2_means_dim'+str(dim)+'x'+str(dim)+ '_pose_'+str(pose)+'_gap_'+str(gap)+'_'+str(trial)+'.png')

## 3-means

In [6]:
def plot_ld1_3means(lda_X,y,y_k3means,dim,pose,gap,trial):
            SMALL_SIZE = 8
            MEDIUM_SIZE = 20
            BIGGER_SIZE = 30

            plt.rc('font', size=BIGGER_SIZE)          # controls default text sizes
            plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
            plt.rc('axes', labelsize=BIGGER_SIZE)    # fontsize of the x and y labels
            plt.rc('xtick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
            plt.rc('ytick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
            plt.rc('legend', fontsize=MEDIUM_SIZE)    # legend fontsize
            plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

            fig, ax = plt.subplots()
            plt.style.use("ggplot")
            plt.rcParams["figure.figsize"] = (12,8)
            ax.axes.yaxis.set_ticks([])
            plt.ylabel('LD1 & 3-means clustering')
            plt.scatter(lda_X[y==0], np.full((len(lda_X[y==0]),),2), color='orange', label='corner')
            plt.scatter(lda_X[y==1], np.full((len(lda_X[y==1]),),2), color='purple', label='side')
            plt.scatter(lda_X[y_k3means==0], np.full((len(lda_X[y_k3means==0]),),1.0), c='red', label ='Cluster 1')
            plt.scatter(lda_X[y_k3means==1], np.full((len(lda_X[y_k3means==1]),),1.0), c='blue', label ='Cluster 2')
            plt.scatter(lda_X[y_k3means==2], np.full((len(lda_X[y_k3means==2]),),1.0), c='green', label ='Cluster 3')
            plt.scatter(k3means.cluster_centers_[:, 0], np.full((len(k3means.cluster_centers_[:, 0]),),1.0), 
                        s=100, c='yellow', marker='x',label = 'Centroids')
            #plt.plot([-3,-3],[0,4],'--',color='g')
            plt.ylim(0,4)
            ax.legend()
            plt.title("LD1 & 3-means: dim="+str(dim)+'x'+str(dim)+ 'mm ; pose='+str(pose)+'mm ; gap='+str(gap)+'mm ; trial='+str(trial),fontsize=20)
            plt.savefig('LD1_3_means_dim'+str(dim)+'x'+str(dim)+ '_pose_'+str(pose)+'_gap_'+str(gap)+'_'+str(trial)+'.png')

            ##########################################################################################

def plot_raw_3means(sides_pos,corners_pos,angle_array,half_window_size,y_k3means,dim,pose,gap,trial):
            SMALL_SIZE = 8
            MEDIUM_SIZE = 20
            BIGGER_SIZE = 30

            plt.rc('font', size=BIGGER_SIZE)          # controls default text sizes
            plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
            plt.rc('axes', labelsize=BIGGER_SIZE)    # fontsize of the x and y labels
            plt.rc('xtick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
            plt.rc('ytick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
            plt.rc('legend', fontsize=MEDIUM_SIZE)    # legend fontsize
            plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
            
            fig, ax = plt.subplots()
            plt.style.use("ggplot")
            plt.rcParams["figure.figsize"] = (12,8)
            plt.scatter(sides_pos.T[0],sides_pos.T[2]+10,
                        color='purple',marker='+',label='sides')
            plt.scatter(corners_pos.T[0],corners_pos.T[2]+10,
                        color='orange',marker='+',label='corners')
            plt.scatter(angle_array[half_window_size:-half_window_size].T[0][y_k3means == 0], 
                        angle_array[half_window_size:-half_window_size].T[2][y_k3means == 0]-10, 
                        marker='+', c='red', label ='Cluster 1')
            plt.scatter(angle_array[half_window_size:-half_window_size].T[0][y_k3means == 1], 
                        angle_array[half_window_size:-half_window_size].T[2][y_k3means == 1]-10, 
                        marker='+', c='blue', label ='Cluster 2')
            plt.scatter(angle_array[half_window_size:-half_window_size].T[0][y_k3means == 2], 
                        angle_array[half_window_size:-half_window_size].T[2][y_k3means == 2]-10, 
                        marker='+', c='green', label ='Cluster 3')
            plt.plot(angle_array[half_window_size:-half_window_size].T[0],
                    angle_array[half_window_size:-half_window_size].T[2],
                    color='yellow', label='raw data')
            plt.ylabel(r'$\theta_R$')
            plt.xlabel(r'$\theta_L$')
            plt.legend(loc='lower right')
            plt.title("raw data processed (3-means): dim="+str(dim)+'x'+str(dim)+ 'mm ; pose='+str(pose)+'mm ; gap='+str(gap)+'mm ; trial='+str(trial),fontsize=20)
            plt.savefig('clusters_class_raw_3_means_dim'+str(dim)+'x'+str(dim)+ '_pose_'+str(pose)+'_gap_'+str(gap)+'_'+str(trial)+'.png')

# Dataset

In [7]:
with open('data_hexagon_trial_classified.json') as f:
    full_data_dict = json.load(f)

poses_drf = [60,65,70]
finger_gaps = [34,42,50]
sqrt_sides = [20,25,30]
trials = [1,2]
f1_scores = {}

for trial in trials:
    for pose in poses_drf:
        for gap in finger_gaps:
            for dim in sqrt_sides:
                dict_key = "dim_"+str(dim)+"_pose_"+str(pose)+"_gap_"+str(gap)+'_'+str(trial)
                data_dict = full_data_dict[dict_key]
    
                data_list = [data_dict['LF_motion1'],data_dict['RF_motion1']]
                data_array = np.array(data_list).T

                temp_data_array = np.append(data_array[np.newaxis,0,:,0],data_array[np.newaxis,1,:,0],axis=0)
                temp_data_array = np.append(temp_data_array,data_array[np.newaxis,0,:,1],axis=0)
                temp_data_array = np.append(temp_data_array,data_array[np.newaxis,1,:,1],axis=0)
                data_array = np.append(temp_data_array,data_array[np.newaxis,2,:,0],axis=0).T
                angle_array = data_array[:,:4].astype(float)
                # CREATE A DATAFRAME BY WINDOWING THE RAW AND INCREMENTING THE POSITION OF THE WINDOW
                rows,columns = data_array.shape
                window_size = 5 # needs to be odd
                half_window_size = math.floor(window_size/2)
                data_window = np.zeros((rows-half_window_size*2,window_size*4+1)) #+1 for the class
                corners = []
                sides = []
                for i in range(half_window_size,rows-half_window_size):
                    temp_window = np.append(data_array[i-half_window_size:half_window_size+i+1,:].T[0],
                                            data_array[i-half_window_size:half_window_size+i+1,:].T[1])
                    temp_window = np.append(temp_window,
                                            data_array[i-half_window_size:half_window_size+i+1,:].T[2])
                    temp_window = np.append(temp_window,
                                            data_array[i-half_window_size:half_window_size+i+1,:].T[3])
                    if 'corner' in data_array[i-half_window_size+1:half_window_size+i+1-1,:].T[4]:
                        # CLASS 2 IS THE CORNER
                        corners += [angle_array[i].tolist()]
                        data_window[i-half_window_size] = np.append(temp_window,np.array([2]))
                    else:
                        # CLASS 1 IS THE SIDE
                        sides += [angle_array[i].tolist()]
                        data_window[i-half_window_size] = np.append(temp_window,np.array([1]))
    
                #print(data_window)
                names = []
                for j in range(window_size):
                    names += ["angleLF"+str(j+1)]
                for j in range(window_size):
                    names += ["timeLF"+str(j+1)]
                for j in range(window_size):
                    names += ["angleRF"+str(j+1)]
                for j in range(window_size):
                    names += ["timeRF"+str(j+1)]
                names += ["class"]

                # DATA FRAME PANDAS
                df = pd.DataFrame(data=data_window,columns = names)
                df["class"].replace({1.0: "side", 2.0: "corner"}, inplace=True)
                df.dropna(how='all', inplace=True)
                windows = df[names[:len(names)-1]]
                labels = df[names[-1]]

                corners_pos = np.array(corners)
                sides_pos = np.array(sides)

                df.head()
                
                ##########################################################################################
                ############################# Plot the positions of the finger ###########################
                ##########################################################################################
                #plot_pos_fingers(sides_pos,corners_pos,dim,pose,gap,trial)

                ##########################################################################################
                ############################# LDA with sklearn ###########################
                ##########################################################################################

                # Preparation of the data for LDA
                X = df.iloc[:, 0:-1].values
                le = LabelEncoder()
                y = le.fit_transform(df['class'])

                # Import LDA from sklearn
                lda = LinearDiscriminantAnalysis()
                lda_X = lda.fit_transform(X, y)
                print(lda_X)
                print("The explained variance ratio of lda is : ", lda.explained_variance_ratio_)

                ##########################################################################################
                ############################# Plot projection on feature vectors ###########################
                ##########################################################################################
                #plot_proj_ld1(lda_X,y,dim,pose,gap,trial)
                
                ##########################################################################################
                #################################### K-means #############################################
                ##########################################################################################

                # 2-means clustering 
                k2means = KMeans(n_clusters=2, init ='k-means++', max_iter=300, n_init=10,random_state=0 )
                y_k2means = k2means.fit_predict(lda_X)

                # 3-mens clustering
                k3means = KMeans(n_clusters=3, init ='k-means++', max_iter=300, n_init=10,random_state=0 )
                y_k3means = k3means.fit_predict(lda_X)

                ##########################################################################################
                #plot_ld1_2means(lda_X,y,y_k2means,dim,pose,gap,trial)
                ##########################################################################################
                #plot_raw_2means(sides_pos,corners_pos,angle_array,half_window_size,y_k2means,dim,pose,gap,trial)
                ##########################################################################################
                #plot_ld1_3means(lda_X,y,y_k3means,dim,pose,gap,trial)
                ##########################################################################################
                #plot_raw_3means(sides_pos,corners_pos,angle_array,half_window_size,y_k3means,dim,pose,gap,trial)
                ##########################################################################################

                ##########################################################################################
                #################################### F1-score ############################################
                ##########################################################################################

                # we need to change "y" before beca
                y_true = []
                for i in range(y.shape[0]):
                    if y[i] == 0:
                        y_true += [1]
                    else:
                        y_true += [0]

                f1_score_2means = f1_score(y_true,y_k2means)
                print("The f1_score of the 2-means is : ",f1_score_2means)
                print(y_true)

                # f1-score on just the corner identification
                y_k3_corner = []
                for i in range(len(y_k3means.tolist())):
                    if y_k3means[i] == 2:
                        y_k3_corner += [1]
                    else :
                        y_k3_corner += [y_k3means[i]]
                y_k3_corner = np.array(y_k3_corner)

                f1_score_3means = f1_score(y_true,y_k3_corner)
                print("The f1_score of the 3-means for global corner identification is : ",f1_score_3means)

                # f1-score on the number of corners
                u = 0
                v = 0
                y_true_corners = []
                for i in range(len(y_true)):
                    if y_true[i] == 1 and v == 0:
                        y_true_corners += [y_true[i]]
                        u = 1
                    elif y_true[i] == 0 and u==1:
                        y_true_corners += [y_true[i]]
                        v = 1
                    elif y_true[i] == 1 and v==1:
                        y_true_corners += [2]
                    else:
                        y_true_corners += [y_true[i]]
                acc_3means = accuracy_score(y_true_corners,y_k3means)
                print("The f1_score of the 3-means for each corner identification (micro/or accuracy) is : ",
                      acc_3means)

                f1_scores[dict_key] = {
                    "The f1_score 2-means corner identification" : f1_score_2means,
                    "The f1_score 3-means global corner identification" : f1_score_3means,
                    "The f1_score 3-means each corner identification" : acc_3means      
                }

[[-0.5806]
 [-0.278 ]
 [ 1.6541]
 [ 1.327 ]
 [-0.5682]
 [-0.2508]
 [ 1.5   ]
 [ 3.0898]
 [ 1.5426]
 [ 1.3104]
 [ 2.0727]
 [ 2.2596]
 [ 0.9685]
 [-0.7403]
 [-0.2354]
 [ 1.5   ]
 [ 1.0633]
 [ 0.0345]
 [ 0.6745]
 [-0.4277]
 [-2.8089]
 [-4.6503]
 [-3.469 ]
 [-2.7229]
 [-2.312 ]
 [-1.8927]
 [-1.203 ]
 [-0.9505]
 [ 0.0588]
 [ 1.125 ]
 [ 0.8078]
 [ 0.9998]
 [ 1.0447]
 [ 0.9736]
 [ 1.0573]
 [ 0.3249]
 [ 1.6482]
 [ 1.9554]
 [ 0.2663]
 [ 0.6073]
 [ 1.7684]
 [ 0.7475]
 [ 0.2705]
 [ 0.0406]
 [ 0.7842]
 [-0.1808]
 [-1.855 ]
 [ 0.1592]
 [ 0.72  ]
 [ 0.2134]
 [ 0.1853]
 [-0.0976]
 [-3.6972]
 [-3.3969]
 [-3.6007]
 [-2.228 ]
 [-1.1687]
 [ 1.1754]
 [ 1.0863]
 [ 1.0822]
 [ 1.2157]]
The explained variance ratio of lda is :  [1.]
The f1_score of the 2-means is :  0.782608695652174
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0]
The f1_score of the 3-means for global corner

The f1_score of the 2-means is :  0.0
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0]
The f1_score of the 3-means for global corner identification is :  0.32258064516129037
The f1_score of the 3-means for each corner identification (micro/or accuracy) is :  0.22950819672131148
[[-1.0385]
 [ 0.0261]
 [ 0.0643]
 [ 1.3548]
 [ 2.6999]
 [ 1.0935]
 [ 1.1086]
 [ 2.1699]
 [ 0.6171]
 [ 0.8058]
 [ 1.3674]
 [ 1.5955]
 [ 2.0932]
 [ 1.8144]
 [ 1.1519]
 [ 1.6419]
 [ 0.8971]
 [ 0.2587]
 [ 0.0534]
 [ 0.5095]
 [-0.3919]
 [-1.2719]
 [-2.1038]
 [-3.001 ]
 [-3.4961]
 [-2.8425]
 [-2.8667]
 [-2.1671]
 [-0.4451]
 [-0.2288]
 [-0.7347]
 [ 0.2306]
 [ 1.9638]
 [ 1.8523]
 [ 0.3013]
 [-0.9109]
 [-0.4561]
 [ 0.3167]
 [ 0.6892]
 [ 0.7257]
 [ 1.8683]
 [ 1.6771]
 [ 0.8627]
 [ 0.4798]
 [ 0.2767]
 [ 0.6164]
 [ 0.899 ]
 [ 0.0745]
 [-0.5834]
 [-0.6957]
 [-0.5645]
 [-0.9267]
 [-1.6701]
 [-

The f1_score of the 2-means is :  0.5555555555555556
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
The f1_score of the 3-means for global corner identification is :  0.5128205128205129
The f1_score of the 3-means for each corner identification (micro/or accuracy) is :  0.6065573770491803
[[ 0.3427]
 [ 2.6729]
 [ 2.6505]
 [ 1.5218]
 [ 0.5763]
 [ 0.863 ]
 [ 0.7576]
 [ 0.7399]
 [ 0.6044]
 [ 1.2923]
 [-0.2623]
 [ 1.1697]
 [-1.2755]
 [ 0.1256]
 [ 0.3824]
 [ 0.9801]
 [-0.1891]
 [-0.458 ]
 [-2.6807]
 [-3.5763]
 [-4.2325]
 [-2.9975]
 [-2.1762]
 [ 0.0468]
 [ 1.2046]
 [-1.6778]
 [-0.2272]
 [ 0.6588]
 [ 0.9123]
 [ 1.8547]
 [ 1.3144]
 [ 1.8586]
 [ 2.249 ]
 [ 1.2815]
 [ 0.8829]
 [ 0.6241]
 [ 0.7977]
 [ 0.2079]
 [ 0.3403]
 [ 0.0413]
 [-0.6706]
 [ 0.4596]
 [-0.1448]
 [ 1.3976]
 [ 0.1057]
 [ 0.7604]
 [-0.626 ]
 [-0.0707]
 [-0.1773]
 [-0.653 ]
 [-1.4817]
 [ 1.1628]
 

The f1_score of the 2-means is :  0.5882352941176471
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
The f1_score of the 3-means for global corner identification is :  0.45000000000000007
The f1_score of the 3-means for each corner identification (micro/or accuracy) is :  0.5737704918032787
[[ 1.1567]
 [ 1.3389]
 [ 2.1541]
 [ 2.0341]
 [ 1.7019]
 [-1.1216]
 [ 1.5843]
 [ 1.4992]
 [ 1.9159]
 [ 1.0745]
 [ 1.1922]
 [-0.5584]
 [ 0.0712]
 [-1.0809]
 [ 0.1825]
 [-0.0601]
 [ 1.0187]
 [-0.8255]
 [-1.3805]
 [-2.2577]
 [-2.8934]
 [-2.8871]
 [-1.6359]
 [-0.0943]
 [-0.0722]
 [-0.7574]
 [-1.9919]
 [-0.4397]
 [-0.0647]
 [ 1.2916]
 [ 0.6414]
 [ 1.3424]
 [ 2.0714]
 [ 1.1487]
 [ 1.8574]
 [ 0.2725]
 [ 0.3583]
 [ 0.5174]
 [-0.1352]
 [ 0.4221]
 [ 0.0591]
 [-0.097 ]
 [ 0.7762]
 [-0.4712]
 [ 0.5896]
 [ 0.7375]
 [ 0.7503]
 [ 0.4758]
 [ 0.7194]
 [-0.1581]
 [-0.5468]
 [-1.3502]


The f1_score of the 2-means is :  0.8333333333333333
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0]
The f1_score of the 3-means for global corner identification is :  0.032786885245901634
The f1_score of the 3-means for each corner identification (micro/or accuracy) is :  0.01639344262295082
[[-0.7656]
 [ 0.8409]
 [ 2.0353]
 [ 2.4963]
 [ 1.1643]
 [ 1.865 ]
 [ 1.7747]
 [ 1.5185]
 [-0.668 ]
 [ 0.1616]
 [ 2.4138]
 [ 2.9198]
 [ 1.3552]
 [-2.2283]
 [-0.2402]
 [ 0.0612]
 [-0.3188]
 [-0.2779]
 [ 0.4476]
 [ 0.8522]
 [ 0.1408]
 [-3.09  ]
 [-3.0363]
 [-3.5074]
 [-2.7659]
 [-2.2936]
 [-1.276 ]
 [ 0.3691]
 [ 0.1038]
 [ 1.408 ]
 [ 0.6181]
 [ 1.6045]
 [-0.2298]
 [-0.2092]
 [ 0.7646]
 [ 1.1474]
 [ 0.7064]
 [ 0.7292]
 [ 1.5258]
 [ 0.0693]
 [ 1.267 ]
 [ 1.2947]
 [ 1.1418]
 [-1.0192]
 [-0.3942]
 [ 0.4458]
 [ 0.2596]
 [-1.1853]
 [-0.6583]
 [ 0.5218]
 [-0.3893]
 [-1.9833

The f1_score of the 2-means is :  0.7826086956521738
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0]
The f1_score of the 3-means for global corner identification is :  0.4
The f1_score of the 3-means for each corner identification (micro/or accuracy) is :  0.4426229508196721
[[-0.1821]
 [-0.7114]
 [ 1.1793]
 [ 2.2217]
 [-0.0734]
 [-0.6179]
 [ 1.4265]
 [ 2.3038]
 [ 1.3958]
 [ 2.1907]
 [ 1.3796]
 [ 1.8154]
 [ 0.8935]
 [-0.2896]
 [ 0.7665]
 [-0.4978]
 [-0.341 ]
 [-0.967 ]
 [-2.103 ]
 [-0.5011]
 [-1.1988]
 [ 0.1805]
 [-3.4532]
 [-5.3388]
 [-6.6541]
 [-5.6741]
 [-4.9286]
 [-1.6003]
 [-1.3236]
 [ 0.7399]
 [-0.2277]
 [ 0.1783]
 [ 1.0154]
 [ 0.9677]
 [ 0.9034]
 [ 0.5986]
 [ 0.9592]
 [ 0.5477]
 [ 0.6414]
 [ 0.707 ]
 [-0.4265]
 [ 1.6766]
 [ 1.2062]
 [ 1.3716]
 [ 1.9182]
 [ 0.8991]
 [-0.3839]
 [ 0.2859]
 [-0.087 ]
 [ 0.1688]
 [-0.4584]
 [-0.3124]
 [ 0.6902]
 [ 0.

The f1_score of the 2-means is :  0.7142857142857143
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0]
The f1_score of the 3-means for global corner identification is :  0.39215686274509803
The f1_score of the 3-means for each corner identification (micro/or accuracy) is :  0.4098360655737705


## BE CAREFUL THIS IS FOR INITIALISATION A JSON FILE

In [9]:
#with open("f1_scores_sqrt_dim_new_w_5_trials.json", 'w') as f:
with open("f1_scores_hexagon.json", 'w') as f:
    indent = 2  # is not needed but makes the file human-readable
    json.dump(f1_scores, f, indent=2)

# Create a table with the result

In [10]:
with open('f1_scores_hexagon.json') as f:
    dict_f1 = json.load(f)

poses_drf = [60,65,70]
finger_gaps = [34,42,50]
sqrt_sides = [20,25,30]
trials = [1,2]

print("\nTABLE 1 \n")
# Print the names of the columns.
print ("{:<12} {:<12} {:<12} {:<12} {:<12} {:<12}".format('DIMENSION', 'POSE', 'GAP', 'F1 2-Means','F1 3-Means','F1 3-Means (each)'))

# print each data item.
for gap in finger_gaps:
    for pose in poses_drf:
        for dim in sqrt_sides:
            for trial in trials:
                dict_key = "dim_"+str(dim)+"_pose_"+str(pose)+"_gap_"+str(gap)+'_'+str(trial)
                dict_value = dict_f1[dict_key]
                f1_2means = round(dict_value['The f1_score 2-means corner identification'],3)
                f1_3means_glob = round(dict_value['The f1_score 3-means global corner identification'],3)
                f1_3means_each = round(dict_value['The f1_score 3-means each corner identification'],3)
                print ("{:<12} {:<12} {:<12} {:<12} {:<12} {:<12}".format(dim,pose,gap,pose,f1_2means, f1_3means_glob, f1_3means_each))


print("\nTABLE 2 \n")
# Print the names of the columns.
print ("{:<12} {:<12} {:<12} {:<12} {:<12} {:<12}".format('DIMENSION', 'POSE', 'GAP', 'F1 2-Means','F1 3-Means','F1 3-Means (each)'))

for pose in poses_drf:
    for gap in finger_gaps:
        for dim in sqrt_sides:
            for trial in trials:
                dict_key = "dim_"+str(dim)+"_pose_"+str(pose)+"_gap_"+str(gap)+'_'+str(trial)
                dict_value = dict_f1[dict_key]
                f1_2means = round(dict_value['The f1_score 2-means corner identification'],3)
                f1_3means_glob = round(dict_value['The f1_score 3-means global corner identification'],3)
                f1_3means_each = round(dict_value['The f1_score 3-means each corner identification'],3)
                print ("{:<12} {:<12} {:<12} {:<12} {:<12} {:<12}".format(dim,pose,gap,pose,f1_2means, f1_3means_glob, f1_3means_each))


print("\nTABLE 3 \n")
# Print the names of the columns.
print ("{:<12} {:<12} {:<12} {:<12} {:<12} {:<12}".format('DIMENSION', 'POSE', 'GAP', 'F1 2-Means','F1 3-Means','F1 3-Means (each)'))

for dim in sqrt_sides:
    for gap in finger_gaps:
        for pose in poses_drf:
            for trial in trials:
                dict_key = "dim_"+str(dim)+"_pose_"+str(pose)+"_gap_"+str(gap)+'_'+str(trial)
                dict_value = dict_f1[dict_key]
                f1_2means = round(dict_value['The f1_score 2-means corner identification'],3)
                f1_3means_glob = round(dict_value['The f1_score 3-means global corner identification'],3)
                f1_3means_each = round(dict_value['The f1_score 3-means each corner identification'],3)
                print ("{:<12} {:<12} {:<12} {:<12} {:<12} {:<12}".format(dim,pose,gap,pose,f1_2means, f1_3means_glob, f1_3means_each))




TABLE 1 

DIMENSION    POSE         GAP          F1 2-Means   F1 3-Means   F1 3-Means (each)
20           60           34           60           0.783        0.439       
20           60           34           60           0.0          0.314       
25           60           34           60           1.0          0.0         
25           60           34           60           0.769        0.286       
30           60           34           60           0.769        0.667       
30           60           34           60           1.0          0.25        
20           65           34           65           0.588        0.417       
20           65           34           65           0.69         0.435       
25           65           34           65           1.0          0.385       
25           65           34           65           0.769        0.323       
30           65           34           65           0.714        0.213       
30           65           34           65       

# Calculate the average f1-score

In [13]:
#with open('f1_scores_sqrt_dim_new_w_5_trials.json') as f:
with open('f1_scores_hexagon.json') as f:
    dict_f1 = json.load(f)

poses_drf = [60,65,70]
finger_gaps = [34,42,50]
sqrt_sides = [20,25,30]
trials = [1,2]

#path = 'f1_scores_sqrt_dim_new_w_5_trials.txt'
path = 'f1_scores_hexagon.txt'
f1_file = open(path,'w')

title = 'All the F1-score and accuracy of the squares simulation for window of size 5 \n'
f1_file.write(title)


################ same size ################ 
f1_file.write('\nSAME SIZE \n\n')
print('\nSAME SIZE \n')

sum_f1_score_2means = 0
sum_f1_score_3means_glob = 0
sum_f1_score_3means_each = 0
nb_f1_Scores = 0

for dim in sqrt_sides:
    for gap in finger_gaps:
        for pose in poses_drf:
            for trial in trials:
                dict_key = "dim_"+str(dim)+"_pose_"+str(pose)+"_gap_"+str(gap)+'_'+str(trial)
                dict_value = dict_f1[dict_key]
                f1_2means = round(dict_value['The f1_score 2-means corner identification'],3)
                f1_3means_glob = round(dict_value['The f1_score 3-means global corner identification'],3)
                f1_3means_each = round(dict_value['The f1_score 3-means each corner identification'],3)
                sum_f1_score_2means += f1_2means
                sum_f1_score_3means_glob += f1_3means_glob
                sum_f1_score_3means_each += f1_3means_each
                nb_f1_Scores += 1
    print('The average f1-score for squares of dim =',str(dim),' and 2-means clustering : ',sum_f1_score_2means/nb_f1_Scores)
    f1_file.write('The average f1-score for squares of dim ='+str(dim)+' and 2-means clustering : '+str(sum_f1_score_2means/nb_f1_Scores)+'\n')
    print('The average f1-score for squares of dim =',str(dim),' and 3-means clustering : ',sum_f1_score_3means_glob/nb_f1_Scores)
    f1_file.write('The average f1-score for squares of dim ='+str(dim)+' and 3-means clustering : '+str(sum_f1_score_3means_glob/nb_f1_Scores)+'\n')
    print('The average accuracy for squares of dim =',str(dim),' and 3-means clustering : ',sum_f1_score_3means_each/nb_f1_Scores,'\n')
    f1_file.write('The average accuracy for squares of dim ='+str(dim)+' and 3-means clustering : '+str(sum_f1_score_3means_each/nb_f1_Scores)+'\n\n')
    sum_f1_score_2means = 0
    sum_f1_score_3means_glob = 0
    sum_f1_score_3means_each = 0
    nb_f1_Scores = 0
    

################ same pose ################ 
print('\nSAME POSE \n')
f1_file.write('\nSAME POSE \n\n')
sum_f1_score_2means = 0
sum_f1_score_2means_glob = 0
sum_f1_score_2means_each = 0
nb_f1_Scores = 0
for pose in poses_drf:
    for dim in sqrt_sides:
        for gap in finger_gaps:
            for trial in trials:
                dict_key = "dim_"+str(dim)+"_pose_"+str(pose)+"_gap_"+str(gap)+'_'+str(trial)
                dict_value = dict_f1[dict_key]
                f1_2means = round(dict_value['The f1_score 2-means corner identification'],3)
                f1_3means_glob = round(dict_value['The f1_score 3-means global corner identification'],3)
                f1_3means_each = round(dict_value['The f1_score 3-means each corner identification'],3)
                sum_f1_score_2means += f1_2means
                sum_f1_score_3means_glob += f1_3means_glob
                sum_f1_score_3means_each += f1_3means_each
                nb_f1_Scores += 1
    print('The average f1-score for squares with pose =',str(pose),' and 2-means clustering : ',sum_f1_score_2means/nb_f1_Scores)
    f1_file.write('The average f1-score for squares with pose ='+str(pose)+' and 2-means clustering : '+str(sum_f1_score_2means/nb_f1_Scores)+'\n')
    print('The average f1-score for squares with pose =',str(pose),' and 3-means clustering : ',sum_f1_score_3means_glob/nb_f1_Scores)
    f1_file.write('The average f1-score for squares with pose ='+str(pose)+' and 3-means clustering : '+str(sum_f1_score_3means_glob/nb_f1_Scores)+'\n')
    print('The average accuracy for squares with pose =',str(pose),' and 3-means clustering : ',sum_f1_score_3means_each/nb_f1_Scores,'\n')
    f1_file.write('The average accuracy for squares with pose ='+str(pose)+' and 3-means clustering : '+str(sum_f1_score_3means_each/nb_f1_Scores)+'\n\n')
    sum_f1_score_2means = 0
    sum_f1_score_3means_glob = 0
    sum_f1_score_3means_each = 0
    nb_f1_Scores = 0


################ same gap ################ 
f1_file.write('\nSAME GAP \n\n')
print('\nSAME GAP \n\n')
sum_f1_score_2means = 0
sum_f1_score_2means_glob = 0
sum_f1_score_2means_each = 0
nb_f1_Scores = 0
for gap in finger_gaps:
    for dim in sqrt_sides:
        for pose in poses_drf:
            for trial in trials:
                dict_key = "dim_"+str(dim)+"_pose_"+str(pose)+"_gap_"+str(gap)+'_'+str(trial)
                dict_value = dict_f1[dict_key]
                f1_2means = round(dict_value['The f1_score 2-means corner identification'],3)
                f1_3means_glob = round(dict_value['The f1_score 3-means global corner identification'],3)
                f1_3means_each = round(dict_value['The f1_score 3-means each corner identification'],3)
                sum_f1_score_2means += f1_2means
                sum_f1_score_3means_glob += f1_3means_glob
                sum_f1_score_3means_each += f1_3means_each
                nb_f1_Scores += 1
    print('The average f1-score for squares with fingers gap =',str(gap),' and 2-means clustering : ',sum_f1_score_2means/nb_f1_Scores)
    f1_file.write('The average f1-score for squares with fingers gap ='+str(gap)+' and 2-means clustering : '+str(sum_f1_score_2means/nb_f1_Scores)+'\n')
    print('The average f1-score for squares with fingers gap =',str(gap),' and 3-means clustering : ',sum_f1_score_3means_glob/nb_f1_Scores)
    f1_file.write('The average f1-score for squares with fingers gap ='+str(gap)+' and 3-means clustering : '+str(sum_f1_score_3means_glob/nb_f1_Scores)+'\n')
    print('The average accuracy for squares with fingers gap =',str(gap),' and 3-means clustering : ',sum_f1_score_3means_each/nb_f1_Scores,'\n')
    f1_file.write('The average accuracy for squares with fingers gap ='+str(gap)+' and 3-means clustering : '+str(sum_f1_score_3means_each/nb_f1_Scores)+'\n\n')
    sum_f1_score_2means = 0
    sum_f1_score_3means_glob = 0
    sum_f1_score_3means_each = 0
    nb_f1_Scores = 0

################ total average ################ 
f1_file.write('\nTOTAL AVERAGE \n\n')
print('\nTOTAL AVERAGE \n\n')
sum_f1_score_2means = 0
sum_f1_score_3means_glob = 0
sum_f1_score_3means_each = 0
nb_f1_Scores = 0
for gap in finger_gaps:
    for dim in sqrt_sides:
        for pose in poses_drf:
            for trial in trials:
                dict_key = "dim_"+str(dim)+"_pose_"+str(pose)+"_gap_"+str(gap)+'_'+str(trial)
                dict_value = dict_f1[dict_key]
                f1_2means = round(dict_value['The f1_score 2-means corner identification'],3)
                f1_3means_glob = round(dict_value['The f1_score 3-means global corner identification'],3)
                f1_3means_each = round(dict_value['The f1_score 3-means each corner identification'],3)
                sum_f1_score_2means += f1_2means
                sum_f1_score_3means_glob += f1_3means_glob
                sum_f1_score_3means_each += f1_3means_each
                nb_f1_Scores += 1
print('The total average f1-score for squares and 2-means clustering : ',sum_f1_score_2means/nb_f1_Scores)
f1_file.write('The total average f1-score for squares and 2-means clustering : '+str(sum_f1_score_2means/nb_f1_Scores)+'\n')
print('The total average f1-score for squares and 3-means clustering : ',sum_f1_score_3means_glob/nb_f1_Scores)
f1_file.write('The total average f1-score for squares and 3-means clustering : '+str(sum_f1_score_3means_glob/nb_f1_Scores)+'\n')
print('The total average accuracy for squares and 3-means clustering : ',sum_f1_score_3means_each/nb_f1_Scores,'\n')
f1_file.write('The total average accuracy for squares and 3-means clustering : '+str(sum_f1_score_3means_each/nb_f1_Scores)+'\n\n')

f1_file.close()


SAME SIZE 

The average f1-score for squares of dim = 20  and 2-means clustering :  0.5087222222222221
The average f1-score for squares of dim = 20  and 3-means clustering :  0.37583333333333335
The average accuracy for squares of dim = 20  and 3-means clustering :  0.3915555555555555 

The average f1-score for squares of dim = 25  and 2-means clustering :  0.6345
The average f1-score for squares of dim = 25  and 3-means clustering :  0.31061111111111106
The average accuracy for squares of dim = 25  and 3-means clustering :  0.4381666666666667 

The average f1-score for squares of dim = 30  and 2-means clustering :  0.6792777777777779
The average f1-score for squares of dim = 30  and 3-means clustering :  0.2959444444444445
The average accuracy for squares of dim = 30  and 3-means clustering :  0.4580000000000001 


SAME POSE 

The average f1-score for squares with pose = 60  and 2-means clustering :  0.5781666666666667
The average f1-score for squares with pose = 60  and 3-means clus

- poor performance for the small square for the 3-means as the window of are overlapping 
- for gap 42 problems

